In [90]:
import pandas as pd
import requests

In [222]:
backups = pd.read_csv("https://raw.githubusercontent.com/kaushik12/DataRescue_jkan/refs/heads/main/baserow_exports/datarescue_backups.csv")
datasets = pd.read_csv("https://raw.githubusercontent.com/kaushik12/DataRescue_jkan/refs/heads/main/baserow_exports/datarescue_datasets.csv")

In [223]:
datasets.dtypes

dataset             object
notes               object
dataset_id           int64
url                 object
websites            object
organization        object
agency              object
last_modified       object
last_modified_by    object
dtype: object

In [230]:
datasets[~(datasets.notes == "")]

,dataset,notes,dataset_id,url,websites,organization,agency,last_modified,last_modified_by
82,National Survey of Mortgage Originations (NSMO),This is a joint collection between the CFPB an...,89,https://www.fhfa.gov/data/national-survey-mort...,"consumerfinance.gov, fhfa.gov","Consumer Financial Protection Bureau, Federal ...","Consumer Financial Protection Bureau, Federal ...",2025-02-20,Kathleen
179,CDC Internet Archive Data File Info,Contains information what data is included in ...,279,https://www.cdc.gov/index.html,cdc.gov,Centers for Disease Control and Prevention,Department of Health and Human Services,2025-03-02,Cataloger
256,National Energy Modeling System (NEMS) input data,Input data to NEMS model. lots of granular rea...,360,https://github.com/EIAgov/NEMS/tree/main/input...,eiagov.github.com,Energy Information Administration,Department of Energy,2025-03-02,Gabrielle
260,Benefits Risk Assessment Health Impacts Screen...,Downloadable program. Web based version and mo...,364,https://www.epa.gov/cobra,cobra.epa.gov,Environmental Protection Agency,Environmental Protection Agency,2025-03-02,Gabrielle
280,National Flood Hazard Layer (NFHL) Viewer,"PDFs of FIRMettes or Full FIRMs, Map images an...",384,https://hazards-fema.maps.arcgis.com/apps/weba...,hazards.fema.gov,Federal Emergency Management Agency,Department of Homeland Security,2025-03-23,Cataloger


In [224]:
datasets.columns = datasets.columns.str.lower()
datasets = datasets.fillna('')
datasets.head()

,dataset,notes,dataset_id,url,websites,organization,agency,last_modified,last_modified_by
0,Billion-Dollar Weather and Climate Disasters,,1,https://www.ncei.noaa.gov/access/billions/mapping,ncei.noaa.gov,National Oceanic and Atmospheric Administration,Department of Commerce,2025-02-10,Cataloger
1,American Communities Survey (ACS),,3,https://www.census.gov/programs-surveys/acs,census.gov,Census Bureau,Department of Commerce,2025-03-03,Cataloger
2,BLS Downloads,,6,https://download.bls.gov,download.bls.gov,Bureau of Labor Statistics,Department of Labor,2025-02-10,Admin
3,CDC FTP,,7,https://ftp.cdc.gov/,ftp.cdc.gov,Centers for Disease Control and Prevention,Department of Health and Human Services,2025-02-10,Admin
4,US Census Bureau FTP,,8,ftp://ftp.census.gov,census.gov,Census Bureau,Department of Commerce,2025-02-11,Admin


In [226]:
backups.columns = backups.columns.str.lower()
backups = backups.fillna('')
backups.head()

,dataset,dataset_id,status,url,source_website,organization,agency,download_date,size,maintainer,download_location,file_type,notes,metadata_available,metadata_url
0,Billion-Dollar Weather and Climate Disasters,1,Finished,https://www.ncei.noaa.gov/access/billions/mapping,ncei.noaa.gov,National Oceanic and Atmospheric Administration,Department of Commerce,2025-02-10,0.15,HD,https://dataverse.harvard.edu/dataset.xhtml?pe...,ZIP,,yes,https://dataverse.harvard.edu/dataset.xhtml?pe...
1,BLS Downloads,6,Finished,https://download.bls.gov,download.bls.gov,Bureau of Labor Statistics,Department of Labor,2025-02-01,47.0,DRP,,,,,
2,CDC FTP,7,Finished,https://ftp.cdc.gov/,ftp.cdc.gov,Centers for Disease Control and Prevention,Department of Health and Human Services,2025-02-01,213.0,DRP,,,,,
3,US Census Bureau FTP,8,Finished,ftp://ftp.census.gov,census.gov,Census Bureau,Department of Commerce,2025-02-01,180.0,DRP,,,"Partial download, server is back online but co...",,
4,National Hurricane Center (NHC),9,Finished,https://www.nhc.noaa.gov/archive,nhc.noaa.gov,NOAA/National Hurricane Center,Department of Commerce,2025-02-06,61.0,DRP,,,,,


In [246]:
backups[backups.metadata_available=="needs review"]

,dataset,dataset_id,status,url,source_website,organization,agency,download_date,size,maintainer,download_location,file_type,notes,metadata_available,metadata_url
596,United States Wind Turbine Database (USWTDB),432,Finished,https://energy.usgs.gov/uswtdb/data/,usgs.gov,U.S. Geological Survey,Department of the Interior,2025-01-31,2.1,CaCo,https://zenodo.org/records/14783215,"Shapefile, CSV",,needs review,
597,United States Large-Scale Solar Photovoltaic D...,433,Finished,https://eerscmap.usgs.gov/uspvdb/data/,usgs.gov,U.S. Geological Survey,Department of the Interior,2025-01-24,0.13,CaCo,https://zenodo.org/records/14736300,"JSON, ZIP",,needs review,


In [234]:
backups.metadata_available.value_counts()

metadata_available
                566
yes             133
no                5
needs review      2
Name: count, dtype: int64

In [256]:
backups[backups.dataset_id == 1].metadata_url.values[0]

'https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/WFMZWP&version=DRAFT'

In [260]:
get_metadata_availability(58)

('Yes', '')

In [351]:
import re
def slugify(string):
    string = clean_text(string)
    # Remove special characters
    string = re.sub(r'[^\w\s-]', '', string)
    # Replace spaces with hyphens
    string = re.sub(r'\s+', '-', string)
    # Convert to lowercase
    string = string.lower()
    return string

def clean_text(string):
    # Remove URL prefixes like http:// or https://
    # string = re.sub(r'http[s]?://', '', string)
    # Remove escape strings like \n
    string = string.replace('\n', '').replace('\r', '').replace('\t', '')
    # Remove leading '-'
    string = re.sub(r'^-', '', string)
    # Replace ':' with '-'
    string = string.replace(':', '')
    return string

In [265]:
backups[backups.dataset_id == 58].index

Index([53, 54, 55, 56], dtype='int64')

In [353]:
def get_metadata_availability(dataset_id):
    """
    This function checks the metadata availability for dataset_id 432 in the backups dataframe.
    It returns "Yes" if metadata is available, "Under Review" if it needs review, and "No" otherwise.
    """
    md_avl = backups[backups.dataset_id == dataset_id].metadata_available.values
    if "yes" in md_avl:

        return "Yes",backups[backups.dataset_id == dataset_id].metadata_url.values[0]
    elif "needs review" in md_avl:
        return "Under Review",""
    else:
        return "No",""

def create_dataset_md(row):
    if row['organization'] == '':
      row['organization'] = 'Unknown'
    ## Defining the schema, filename and path
    schema = 'data_rescue_project'
    dataset_filename = slugify(row['dataset'])
    dataset_path = "../_datasets"
    org_filename = slugify(row['organization'])
    org_path = "../_organizations"

    ## Get backups for each dataset
    data_backups = backups[backups.dataset == row['dataset']]
    metadata_available, metadata_url = get_metadata_availability(row['dataset_id'])
    ## Creating the dataset markdown file
    ## Dataset-level information
    dataset_md = "---\n"
    dataset_md += f"schema: {schema} \n"
    dataset_md += f"title: {clean_text(row['dataset'])}\n"
    dataset_md += f"organization: {clean_text(row['organization'])}\n"
    dataset_md += f"agency: {clean_text(row['agency'])}\n"
    dataset_md += f"websites: {row['websites']}\n"
    dataset_md += f"data_source: {row['url']}\n"
    dataset_md += f"description: {clean_text(row['notes'])}\n"
    dataset_md += f"last_modified: {row['last_modified']}\n"
    dataset_md += f"last_modified_by: {row['last_modified_by']}\n"
    ## Check if any backups have metadata available and populate
    dataset_md += f"metadata_available: {metadata_available}\n"
    dataset_md += f"metadata_url: {metadata_url}\n"
    # dataset_md += f"category: {clean_text(row['notes'])}\n" # to add category

    dataset_md += f"resources:\n"
    ## Resource-level information
    for index, backup_row in data_backups.iterrows():
      dataset_md += f"  - id: {index}\n"
      dataset_md += f"    url: {backup_row['download_location']}\n"
      dataset_md += f"    format: {clean_text(backup_row['file_type'])}\n"
      dataset_md += f"    status: {clean_text(backup_row['status'])}\n"
      dataset_md += f"    size: {backup_row['size']}\n"
      dataset_md += f"    download_date: {backup_row['download_date']}\n"
      dataset_md += f"    maintainer: {clean_text(backup_row['maintainer'])}\n"
      dataset_md += f"    notes: {clean_text(backup_row['notes'])}\n"
    dataset_md += "---\n"
      
    ## Writing the dataset markdown file
    with open(f'{dataset_path}/{dataset_filename}.md', 'w') as output:
      output.write(dataset_md)
    
    ## Creating the organization markdown file
    org_md = "---\n"
    org_md += f"title: {clean_text(row['organization'])} \n" 
    org_md += f"description: \n" 
    org_md += "---\n"

    ## Writing the organization markdown file
    with open(f'{org_path}/{org_filename}.md', 'w') as output:
      output.write(org_md)

In [344]:
backups[backups.index==245]

,dataset,dataset_id,status,url,source_website,organization,agency,download_date,size,maintainer,download_location,file_type,notes,metadata_available,metadata_url
245,AIDS Drug Assistance Programs (ADAPS),273,Finished,https://www.hrsa.gov/grants/find-funding/HRSA-...,hrsa.gov,Health Resources and Services Administration,Department of Health and Human Services,2025-01-31,,ICPSR,https://www.dropbox.com/scl/fo/60drbfxp3p1hb1l...,CSV,,,


In [330]:
text = 'Archive link is to Part 1.  Will include three more parts with url format:\r\n\r\nhttps://archive.org/details/international-comprehensive-ocean-atmosphere-{part number}'

In [331]:
clean_text(text)

'Archive link is to Part 1.  Will include three more parts with url formatarchive.org/details/international-comprehensive-ocean-atmosphere-{part number}'

In [354]:
datasets.apply(create_dataset_md, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
698    None
699    None
700    None
701    None
702    None
Length: 703, dtype: object